## Mediator

A component that facilitates communication between other components without them necessarily being aware of each other or having direct (reference) 
access to each other.

In [2]:
class Person:
    def __init__(self, name):
        self.name = name
        self.chat_log = []
        self.room = None
        
    def receive(self, sender, message):
        s = f"{sender}: {message}"
        print(f"[{self.name}'s chat session] {s}")
        self.chat_log.append(s)
        
    def private_message(self, who, message):
        self.room.message(self.name, who, message)
        
    def say(self, message):
        self.room.broadcast(self.name, message)
        
    
class CharRoom: # <- mediator
    def __init__(self):
        self.people = []
        
    def join(self, person):
        join_msg = f"{person.name} joins the chat"
        self.broadcast("room", join_msg)
        person.room = self
        self.people.append(person)
    
    def broadcast(self, source, message):
        for p in self.people:
            if p.name != source:
                p.receive(source, message)
            
    def message(self, source, destination, message):
        for p in self.people:
            if p.name == destination:
                p.receive(source, message)
                

room = CharRoom()

john = Person('John')
jane = Person('Jane')
room.join(john)
room.join(jane)

john.say('hi room!')
jane.say('oh, hey john')

simon = Person('Simon')
room.join(simon)
simon.say('hi everyone')

jane.private_message('Simon', 'glad you could join us!')

[John's chat session] room: Jane joins the chat
[Jane's chat session] John: hi room!
[John's chat session] Jane: oh, hey john
[John's chat session] room: Simon joins the chat
[Jane's chat session] room: Simon joins the chat
[John's chat session] Simon: hi everyone
[Jane's chat session] Simon: hi everyone
[Simon's chat session] Jane: glad you could join us!


### Mediator with Events

In [3]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)
            
class Game:
    def __init__(self):
        self.events = Event()
        
    def fire(self, args):
        self.events(args)
        
class GoalScoredInfo:
    def __init__(self, who_scored, goals_scored):
        self.goals_scored = goals_scored
        self.who_scored = who_scored
        
class Player:
    def __init__(self, name, game):
        self.game = game
        self.name = name
        self.goals_scored = 0
        
    def score(self):
        self.goals_scored += 1
        args = GoalScoredInfo(self.name, self.goals_scored)
        self.game.fire(args)
        
class Coach:
    def __init__(self, game):
        game.events.append(self.celebrate_goal)
        
    def celebrate_goal(self, args):
        if isinstance(args, GoalScoredInfo) and \
            args.goals_scored < 3:
            print(f"Coach says: well done, {args.who_scored}!")
            
game = Game()
player = Player("Sam", game)
coach = Coach(game)
player.score()
player.score()
player.score()

Coach says: well done, Sam!
Coach says: well done, Sam!
